In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(tidyverse)
    library(data.table)
    library(googlesheets)
}))


In [2]:
phewas_all <- '/oak/stanford/groups/mrivas/projects/biomarkers/phewas/v2_imp/ukb24983_imp_v3.glm.logistic.hybrid.gz'

phewas_hits <- '/oak/stanford/groups/mrivas/projects/biomarkers/phewas/v2_imp/ukb24983_imp_v3.phewas.hits.tsv'


In [3]:
phewas_df <- fread(cmd=paste('zcat', phewas_all)) %>% 
rename('CHROM' = '#CHROM')


In [4]:
phewas_df %>% select(GBE_ID) %>% unique() %>% pull() %>% length()

[1] 172

In [5]:
phewas_df %>% 
filter(
    (CHROM != 6) |
    (CHROM == 6 & POS < 25477797) | 
    (CHROM == 6 & 36448354 < POS)
) %>%
select(ID) %>% unique() %>% pull() %>% length()

[1] 43

In [6]:
0.05 / (172 * 43)

[1] 6.760411e-06

In [7]:
gs_auth(token = "/home/users/ytanigaw/.googlesheets_token.rds")

GBE_names_df <- 'https://docs.google.com/spreadsheets/d/1gwzS0SVZBSKkkHgsoqB5vHo5JpUeYYz8PK2RWrHEq3A' %>%
gs_url() %>% gs_read(ws = 'GBE_names')


Auto-refreshing stale OAuth token.
Sheet-identifying info appears to be a browser URL.
googlesheets will attempt to extract sheet key from the URL.
Putative key: 1gwzS0SVZBSKkkHgsoqB5vHo5JpUeYYz8PK2RWrHEq3A
Warning message:
“`data_frame()` is deprecated as of tibble 1.1.0.
Please use `tibble()` instead.
This warning is displayed once every 8 hours.
Call `lifecycle::last_warnings()` to see where this warning was generated.”Sheet successfully identified: "GBE_names"
Warning message:
“`as_data_frame()` is deprecated as of tibble 2.0.0.
Please use `as_tibble()` instead.
The signature and semantics have changed, see `?as_tibble`.
This warning is displayed once every 8 hours.
Call `lifecycle::last_warnings()` to see where this warning was generated.”Accessing worksheet titled 'GBE_names'.
Parsed with column specification:
cols(
  GBE_category = col_character(),
  GBE_ID = col_character(),
  GBE_N = col_double(),
  GBE_NAME = col_character(),
  GBE_short_name = col_character(),
  GBE_short_na

In [8]:
phewas_hits_df <- phewas_df %>% filter(
    P < 1e-7,
    (CHROM != 6) |
    (CHROM == 6 & POS < 25477797) | 
    (CHROM == 6 & 36448354 < POS)    
)  %>%
select(-REF, -ALT, -A1, -'FIRTH?', -TEST, -ERRCODE) %>%
left_join(
    GBE_names_df %>% select(GBE_ID, GBE_short_name), 
    by='GBE_ID'
)

In [9]:
phewas_hits_df %>% dim()

[1] 14 10

In [10]:
phewas_hits_df %>% select(ID) %>% pull() %>% unique() %>% length()


[1] 8

In [11]:
phewas_hits_df %>% select(GBE_ID) %>% pull() %>% unique() %>% length()


[1] 11

In [12]:
phewas_hits_df

CHROM,POS,ID,OBS_CT,OR,LOG(OR)_SE,Z_STAT,P,GBE_ID,GBE_short_name
<int>,<int>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
1,152285861,1:152285861:G:A,307048,1.265330,0.01971920,11.93410,7.86192e-33,BIN_FC5006152,Hayfever rhinitis or eczema diagnosed by doctor
1,152285861,1:152285861:G:A,337146,1.229250,0.02384610,8.65571,4.89842e-18,HC382,Asthma
1,152285861,1:152285861:G:A,337146,1.915950,0.03621370,17.95490,4.39627e-72,HC261,Eczema/dermatitis
1,156784982,1:156784982:T:C,337146,0.930486,0.01097840,-6.56273,5.28301e-11,HC219,Hypothyroidism/myxoedema
6,7231843,6:7231843:G:A,337146,0.873727,0.01735970,-7.77589,7.49185e-15,HC221,Diabetes
8,145158607,8:145158607:G:A,307048,1.066280,0.01190740,5.38925,7.07523e-08,BIN_FC5006152,Hayfever rhinitis or eczema diagnosed by doctor
8,145158607,8:145158607:G:A,337146,1.087570,0.01456460,5.76368,8.22972e-09,HC382,Asthma
10,64564934,10:64564934:C:G,337146,0.969296,0.00549627,-5.67396,1.39531e-08,HC215,Hypertension
12,121416650,12:121416650:A:C,337146,0.916211,0.01401720,-6.24299,4.29273e-10,HC225,Cholecystitis


In [13]:
phewas_hits_df %>% fwrite(phewas_hits, sep='\t')